# Pre-processing of time-series

Time series are measurements that are indexed according to time. For example, electrophysiological recordings or live fluorescence signals. Depending on the goal and the recording, pre-processing or processing of time series data might include: detrending, filtering and/or smoothing. 

Save each step/trace separately, and save the parameters applied to each trace. For example, save them as [Numpy file](https://numpy.org/doc/stable/reference/generated/numpy.save.html).

This notebook contains some simple functions to test and compare different ways for pre-processing time series. The links below contain more information and resources.

**Time-series**
* [A Very Short Course on Time Series Analysis](https://bookdown.org/rdpeng/timeseriesbook/) by Roger D. Peng.

**Filtering**
* [Filtering EEG Data](https://neuraldatascience.io/7-eeg/erp_filtering.html?highlight=power+spectra) from [Neural Data Science in Python](https://neuraldatascience.io/intro.html) by Aaron J. Newman.
* [Filtering Field Data](https://mark-kramer.github.io/Case-Studies-Python/06.html) from [Case studies in Neural Data Analysis](https://mark-kramer.github.io/Case-Studies-Python/intro.html) by Mark Kramer and Uri Eden.
* [Signal processing](https://github.com/BIPN162/Materials/blob/master/15-SignalProcessing.ipynb) from [Neural Data Science course](https://github.com/BIPN162/Materials/tree/master) by Ashley Juavinett.

**Smoothing**
* [A brief introduction to time series smoothing](https://medium.com/@jodancker/a-brief-introduction-to-time-series-smoothing-4f7ed61f78e1) by Jonte Dancker. 


# Example data

Time-series data from widefield recordings of fluorescent sensors. 


# Import the packages

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os

# Interactive plots (comment out if needed)
# import ipywidgets as widgets
# from IPython.display import display
# #For Jupyter Lab:
# %matplotlib widget

# Scipy libraries

from scipy import signal, stats
from scipy.signal import find_peaks
from scipy.optimize import curve_fit
from scipy.ndimage import gaussian_filter1d

from statsmodels.nonparametric.smoothers_lowess import lowess

import sys

# Paths

Change the paths and folder names according to your data structure

In [ ]:
notebook_name = 'time_series_preprocessing'

# Data path to 'Data_example' folders. Change accordingly to your data structure.
data_path = os.path.dirname(os.getcwd())  # Moves one level up from the current directory

# Change the folder names accordingly
paths = {'data': data_path,
         'raw_data':  f'{data_path}/Data_examples/{notebook_name}/',
         'processed_data': f'{data_path}/Processed_data_examples/{notebook_name}/',
         'analysis': f'{data_path}/Analysis_examples/{notebook_name}/',         
         'plots': f'{data_path}/Analysis_examples/{notebook_name}/Plots/'}

# Make folders if they do not exist yet
for path in paths.values():
    os.makedirs(path, exist_ok=True)

# Functions

## Detrend

In detrending, we try to remove some aspects that are distorting the signal and we assume (or evaluate) are not part of our biological variable of interest. For example: rundown due to intracellular solution, photobleaching, baseline instability, etc. Consider that the applied detrend method has to be logically related to the external confound you would like to remove. 

As example, this function applies some commonly detrend methods for electrophysiology and fluorescent sensor traces. From subtracting the average because the baseline is not zero, to non-linear fitting for unstable baselines. Simplify or extend the below function according to your analysis. Documentation:

* Subtract the average: E.g. `x - np.mean(x)`, median, median of lowest values, etc
* [Scipy linear detrend](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html)
* [Numpy Differencing](https://numpy.org/doc/stable/reference/generated/numpy.diff.html)
* [Scipy exponential decay](https://docs.scipy.org/doc/scipy/reference/generated/scipy.optimize.curve_fit.html) 
* Model fitting (polynomial): https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html

In [ ]:
def timeseries_detrend(trace_data, timestamps, method='exponential', window=None):
    """
    Detrend a time-series trace using linear, exponential, polynomial fit, rolling linear correction, median subtraction or lowess.
        - Scipy linear detrend: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html
        - Scipy exponential fit: https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.detrend.html
        - Numpy polynomial fit: https://numpy.org/doc/stable/reference/generated/numpy.polyfit.html
        - Rolling baseline correction with scipy linear regression: https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.linregress.html
        - Numpy median subtraction: https://numpy.org/doc/stable/reference/generated/numpy.median.html
        - Locally Weighted Scatterplot Smoothing: https://www.statsmodels.org/devel/generated/statsmodels.nonparametric.smoothers_lowess.lowess.html

    Args:
        trace_data (numpy array): The data trace to be detrended.
        timestamps (numpy array): The time array corresponding to the trace.
        method (str): Detrending methods ('linear', 'exponential', 'polyfit', 'rolling_linear', 'median', 'lowess').
        window (int, optional): The window size for rolling linear correction or other methods (optional).

    Returns:
        numpy array: Detrended trace.
    """

    # Convert pandas Series to numpy array if needed
    if isinstance(trace_data, pd.Series):
        trace_data = trace_data.to_numpy()
        timestamps = timestamps.to_numpy()
    
    if method == 'linear':
        if window:
            return np.concatenate([
                signal.detrend(trace_data[i:i + window], type='linear') 
                for i in range(0, len(trace_data), window)
            ])
        else:
            return signal.detrend(trace_data, type='linear')
        
    elif method == 'exponential':
        if window:
            exp_detrend_trace = []
            for i in range(0, len(trace_data), window):
                trace_segment = trace_data[i:i + window]
                time_segment = timestamps[i:i + window]
                initial_a = 20  
                initial_b = 0.005
                popt, _ = curve_fit(lambda t, a, b: a * np.exp(-t * b), 
                                    time_segment, trace_segment, 
                                    p0=(initial_a, initial_b))
                a, b = popt
                exp_fit = a * np.exp(-time_segment * b)
                exp_detrend_trace.append(trace_segment - exp_fit)
            return np.concatenate(exp_detrend_trace)
        
        else:
            initial_a = 20  
            initial_b = 0.005  
            popt, _ = curve_fit(lambda t, a, b: a * np.exp(-t * b), 
                                timestamps, trace_data, 
                                p0=(initial_a, initial_b))
            a, b = popt
            exp_fit = a * np.exp(-timestamps * b)
            exp_detrend_trace = trace_data - exp_fit
            return exp_detrend_trace
    
    elif method == 'polyfit':
        if window:
            polyfit_detrend = []
            for i in range(0, len(trace_data), window):
                trace_segment = trace_data[i:i + window]
                time_segment = timestamps[i:i + window]
                polyfit = np.poly1d(np.polyfit(time_segment, trace_segment, 6))
                polyfit_len = polyfit(np.linspace(0, time_segment[-1], len(trace_segment)))
                polyfit_detrend.append(trace_segment - polyfit_len)
            return np.concatenate(polyfit_detrend)
        else:
            polyfit = np.poly1d(np.polyfit(timestamps, trace_data, 6))
            polyfit_len = polyfit(np.linspace(0, timestamps[-1], len(trace_data)))
            polyfit_detrend_trace = trace_data - polyfit_len
            return polyfit_detrend_trace

    elif method == 'rolling_linear':
        if window is None:
            raise ValueError("Provide value for 'window'")
        n = np.floor(len(trace_data) / window)
        fit_trace = trace_data[: int(n * window)]
        x_bin = np.arange(0, len(fit_trace), window) + window / 2
        folded_trace = fit_trace.reshape((window, -1), order='F')
        medians = np.median(folded_trace, axis=0)
        a, b = stats.linregress(x_bin, medians)[:2]
        rolling_linear_trace = trace_data - (a * np.arange(len(trace_data)) + b)
        return rolling_linear_trace
    
    elif method == 'median':
        median_value = np.median(trace_data)
        median_detrend_trace = trace_data - median_value
        return median_detrend_trace
    
    elif method == 'lowess':
        # Apply LOESS/LOWESS smoothing
        smoothed = lowess(trace_data, timestamps, frac=window/np.ptp(timestamps))
        trend = smoothed[:, 1]
        return trace_data - trend
    
    else:
        raise ValueError("Choose a valid 'method'")

## Filtering

A filter operates on digitized data to either pass or reject a defined frequency range. All filters distort the signal to some degree, and each one has different attenuation limitations and artifacts.

Common filters for **time-domain** biological data include:
* **Bessel** filter. [Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.bessel.html) 
* **Gaussian** filter. [Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.ndimage.gaussian_filter1d.html)

Common filters for **frequency-domain** applications include:
* **Butterworth** filter (non-constant delay characteristics). [Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.butter.html).

Apply the filter to the signal: `filtfilt` (zero-phase filtering). [Documentation](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.filtfilt.html)

**References** 
* Widmann et al., 2015. [Digital filter design for electrophysiological data – a practical approach](https://www.sciencedirect.com/science/article/pii/S0165027014002866)
* When, Why, and How (Not) to Use Them. [Cheveigne and Nelken, 2019](https://doi.org/10.1016/j.neuron.2019.02.039)

In [ ]:
def timeseries_filtering(trace_data, fs, filter_type='highpass', filter_order=4, filter_cutoff=1, method='butter'):
    """
    Apply a Bessel or Butterworth filter to a time-series trace.

    Args:
        trace_data (numpy array or pandas Series): The data trace to be filtered. Can also be a pandas Series.
        fs (float): Sampling frequency of the signal.
        filter_type (str): Type of the filter ('lowpass', 'highpass', 'bandpass', 'bandstop').
        filter_order (int): Order of the filter.
        filter_cutoff (float or tuple): Cutoff frequency of the filter. For bandpass and bandstop filters, provide a tuple of (low, high).
        method (str): Filter method ('bessel' or 'butter').

    Returns:
        numpy array: Filtered trace.
    """
    # Convert pandas Series to numpy array if needed
    if isinstance(trace_data, pd.Series):
        trace_data = trace_data.to_numpy()
    
    if method == 'bessel':
        # Design Bessel filter
        b, a = signal.bessel(
            filter_order,         # Order of the filter
            filter_cutoff,        # Cutoff frequency
            filter_type,          # Type of filter
            analog=False,         # Analog or digital filter
            norm='phase',         # Frequency normalization
            fs=fs                 # Sampling frequency
        )
    elif method == 'butter':
        # Design Butterworth filter
        b, a = signal.butter(
            filter_order,         # Order of the filter
            filter_cutoff,        # Cutoff frequency
            filter_type,          # Type of filter
            fs=fs                 # Sampling frequency
        )
    else:
        raise ValueError("Choose a 'method': 'bessel' or 'butter'")

    # Apply the filter
    filtered_trace = signal.filtfilt(b, a, trace_data)

    return filtered_trace

## Padding

In [ ]:
def timeseries_padding(trace_data, window_size):
    """
    Apply padding to a Pandas Series or numpy array to handle edge effects for rolling operations.
    
    Args:
        trace_data (pd.Series or np.ndarray): The data series or array to pad.
        window_size (int): The size of the rolling window.
    
    Returns:
        pd.Series or np.ndarray: Padded data series or array.
    """
    # Convert to pandas Series if trace_data is numpy array
    if isinstance(trace_data, np.ndarray):
        trace_data = pd.Series(trace_data)
    
    # Apply padding
    pad_width = window_size // 2
    trace_padded = pd.concat([
        pd.Series([trace_data.iloc[0]] * pad_width),
        trace_data,
        pd.Series([trace_data.iloc[-1]] * pad_width)
    ], ignore_index=True)
    
    # Convert back to numpy array if the input was a numpy array
    if isinstance(trace_data, np.ndarray):
        return trace_padded.to_numpy()

    return trace_padded

# Detrend time-series data: plots

Note that the LOESS/LOWESS method can be too aggressive for signals with low SNR or slow dynamics, and may remove information from the signal.

In [ ]:
# Define experiments to analyze
experiments = ("WF01", "WF02")

# Assuming paths['raw_data'] is the root directory for the files
for dirpath, dirnames, filenames in os.walk(paths['raw_data']):
    for filename in filenames:
        # CSV files from specific experiments
        if filename.startswith(experiments) and filename.endswith(".csv"):
            file_path = os.path.join(dirpath, filename)
            basename = os.path.splitext(filename)[0]
            timeseries = pd.read_csv(file_path)
            timestamps = timeseries.iloc[:, 0]  # Select column with time/frame information
            trace_data = timeseries.iloc[:, 1]  # Select column with signal information

            window = int(np.round(len(trace_data)/10))

            # Linear detrend
            timeseries_linear_detrend = timeseries_detrend(trace_data, timestamps, method='linear', window=None)

            # Exponential detrend
            timeseries_exponential_detrend = timeseries_detrend(trace_data, timestamps, method='exponential', window=None)

            # Rolling linear
            timeseries_linregress_detrend = timeseries_detrend(trace_data, timestamps, method='rolling_linear', window=window)

            # LOESS/LOWESS
            timeseries_lowess_detrend = timeseries_detrend(trace_data, timestamps, method='lowess', window=window)
            
            # Plotting
            fig, (ax1, ax2, ax3, ax4) = plt.subplots(4, 1, figsize=(8, 8))

            # Labels for all plots
            axes = [ax1, ax2, ax3, ax4]
            for ax in axes:
                ax.set_xlabel("Timestamps")
                ax.set_ylabel("Signal")

            # ax1: Linear detrending
            ax1.plot(timestamps, trace_data, label=f"{basename} raw trace")
            ax1.plot(timestamps, timeseries_linear_detrend, label='Linear detrend')
            
            # Apply linear regression
            slope, intercept, _, _, _ = stats.linregress(timestamps, trace_data)
            linear_fit = slope * timestamps + intercept
            ax1.plot(timestamps, linear_fit, label='Linear Regression', color='black', linestyle='--')
            ax1.legend(loc='upper right')

            # ax2: Exponential detrending
            ax2.plot(timestamps, trace_data, label=f"{basename} raw trace")
            ax2.plot(timestamps, timeseries_exponential_detrend, label='Exponential detrend')
            
            # Plot exponential fit
            def exp_func(t, a, b):
                return a * np.exp(-t * b)
            popt, _ = curve_fit(exp_func, timestamps, trace_data, p0=(20, 0.005))
            a, b = popt
            exp_fit = exp_func(timestamps, a, b)
            ax2.plot(timestamps, exp_fit, label='Exponential Fit', color='black', linestyle='--')
            ax2.legend(loc='upper right')

            # ax3: Rolling linear regression
            ax3.plot(timestamps, trace_data, label=f"{basename} raw trace")
            ax3.plot(timestamps, timeseries_linregress_detrend, label='Rolling Linear Detrend')
            
            # Compute and plot regression lines for each time window
            n = len(trace_data)
            for start in range(0, n, window):
                end = min(start + window, n)
                time_segment = timestamps[start:end]
                trace_segment = trace_data[start:end]
                if len(time_segment) > 1 and len(trace_segment) > 1:
                    slope, intercept, _, _, _ = stats.linregress(time_segment, trace_segment)
                    reg_line = slope * timestamps[start:end] + intercept
                    ax3.plot(timestamps[start:end], reg_line, color='black', linestyle='--')
            ax3.legend(loc='upper right')
       
            # ax4: LOESS/LOWESS
            ax4.plot(timestamps, trace_data, label=f"{basename} raw trace")
            ax4.plot(timestamps, timeseries_lowess_detrend, label='LOESS/LOWESS Detrend')
    
            # Plot LOESS/LOWESS fit
            timeseries_np = np.vstack((timestamps, trace_data)).T
            smoothed = lowess(trace_data, timestamps, frac=window/np.ptp(timeseries_np[:, 0]))
            ax4.plot(smoothed[:, 0], smoothed[:, 1], color='black', linestyle='--', label='LOESS/LOWESS Fit')
            ax4.legend(loc='upper right')
            
            # Show the plots
            plt.tight_layout()

            # Save plots
            fig.savefig(f"{paths['plots']}/{filename}_detrending_examples.tif", dpi=300)  # use '.svg' to save as vector image.
            plt.show()

# Power spectra: plots

Power spectrum analysis applies a fast Fourier transform (FFT) to the variation of a particular signal to compute its frequency spectrum. FFT is based on the principle that any continuous periodic signal can be represented as the sum of sine waves, and vice versa.

The FFT estimates the amplitude of each sine wave (the power) and can be plotted in the frequency domain. This plot is called power spectrum (RMS units) or power spectral density (PSD). In PSD, RMS units are divided by Hz. PSD can be calculated by different approaches such as periodogram or Welch method.

Calculating the PSD is normally more useful for detecting noise ephys traces, but slow artifacts can also be present in slow optical recordings. This information is useful for properly filtering the signal afterward.

* Scipy power spectral density. [Periodogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.periodogram.html) and [Welch](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.welch.html#scipy.signal.welch) 

* [Scipy spectogram](https://docs.scipy.org/doc/scipy/reference/generated/scipy.signal.spectrogram.html)

**Note**: Remember to sample the signal at least twice its fastest frequency component of interest ([Nyquist criterion](https://en.wikipedia.org/wiki/Nyquist_rate)) 

In [ ]:
# Loop through files
for dirpath, dirnames, filenames in os.walk(paths['raw_data']):
    for filename in filenames:
        
        # CSV files from specific experiments
        if filename.startswith(experiments) and filename.endswith(".csv"):
            basename = os.path.splitext(filename)[0]
            file_path = os.path.join(dirpath, filename)
            timeseries = pd.read_csv(file_path)
            timestamps = timeseries.iloc[:, 0]  # Select column with time/frame information
            trace_data = timeseries.iloc[:, 1]  # Select column with signal information

            # Apply detrend. For example: LOESS/LOWESS 
            trace_detrended = timeseries_detrend(trace_data, timestamps, method='lowess', window=300)
            recording_length_s = 300
            fs = len(trace_detrended)/recording_length_s
            f, Pxx_den = signal.periodogram(trace_detrended, fs)
            fig, ax = plt.subplots(figsize=(6,2))
            ax.semilogy(f[1:], Pxx_den[1:])
            ax.set_title(f"{basename}")
            ax.set_xlabel('Frequency (Hz)')
            ax.set_ylabel('PSD ($V^2$/Hz)')

            # Save plots
            fig.savefig(f"{paths['plots']}/{filename}_power_spectra.tif", dpi=300)  # use '.svg' to save as vector image.
            plt.show()

# Filter time-series data: plots

In [ ]:
# Loop through files
for dirpath, dirnames, filenames in os.walk(paths['raw_data']):
    for filename in filenames:
        
        # CSV files from specific experiments
        if filename.startswith(experiments) and filename.endswith(".csv"):
            basename = os.path.splitext(filename)[0]
            file_path = os.path.join(dirpath, filename)
            timeseries = pd.read_csv(file_path)
            timestamps = timeseries.iloc[:, 0]  # Select column with time/frame information
            trace_data = timeseries.iloc[:, 1]  # Select column with signal information

            # Apply detrend. For example: LOESS/LOWESS 
            trace_detrended = timeseries_detrend(trace_data, timestamps, method='lowess', window=300)  

            # Define sampling frequency
            recording_length_s = 300
            fs = len(timestamps)/recording_length_s 

            # Apply filters
            # Bessel filter
            trace_bessel_filter = timeseries_filtering(trace_detrended, fs, filter_type='lowpass', 
                                                       filter_order=4, filter_cutoff=fs/3, method='bessel')
            
            # Gaussian filter
            trace_gaussian_filter = gaussian_filter1d(trace_detrended, sigma=3) 
            
            # Butterworth filter
            trace_butter_filter = timeseries_filtering(trace_detrended, fs, filter_type='lowpass', 
                                                       filter_order=4, filter_cutoff=fs/3, method='butter')

            # Plotting
            fig, (ax1, ax2, ax3, ax4) = plt.subplots(1, 4, figsize=(12, 3), sharey=True)

            # Labels for all plots
            axes = [ax1, ax2, ax3, ax4]
            for ax in axes:
                ax.set_xlabel("Timestamps")
                ax.set_ylabel("Signal")
            
            ax1.set_title(f"{basename}{' Detrended'}")
            ax1.plot(timestamps, trace_detrended)

            ax2.set_title(f"{basename}{' Bessel filter'}")
            ax2.plot(timestamps, trace_bessel_filter)
            
            ax3.set_title(f"{basename}{' Gaussian filter'}")
            ax3.plot(timestamps, trace_gaussian_filter)
            
            ax4.set_title(f"{basename}{' Butterworth filter'}")
            ax4.plot(timestamps, trace_butter_filter)

            # Show the plots
            plt.tight_layout()

            # Save plots
            fig.savefig(f"{paths['plots']}/{filename}_filtering_examples.tif", dpi=300)
            plt.show()

# Smooth time-series data: plots

Smoothing removes certain frequencies or noise of a time series to get the general trend. Basically, it averages points with their neighbors in a time series. Two of the most common approaches are:

* Moving average. Values within the sliding window are averaged and slide through the [time-series pandas rolling](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.rolling.html). 
* Kernel smoothing. Instead of a time window, weights of the Kernel function are used for averaging the sliding window. It can be implemented using pandas `rolling` and Scipy `win_type`. Different kernel functions lead to different results. For example, a kernel with a shape of a gaussian. [Documentation](https://docs.scipy.org/doc/scipy/reference/signal.windows.html).

Since you need to apply a time window for smoothing, there will be missing data at the beginning and end of your time series due to the rolling window operations. To fix this, you can apply padding to ensure that these edges are handled appropriately.


In [ ]:
# Define window size and other parameters
window_size = 10
std = 10  # Standard deviation for Gaussian window

# Loop through files
for dirpath, dirnames, filenames in os.walk(paths['raw_data']):
    for filename in filenames:
        
        # CSV files from specific experiments
        if filename.startswith(experiments) and filename.endswith(".csv"):
            file_path = os.path.join(dirpath, filename)
            timeseries = pd.read_csv(file_path)
            timestamps = timeseries.iloc[:, 0]  # Select column with time/frame information
            trace_data = timeseries.iloc[:, 1]  # Select column with signal information
            
            # Convert to Pandas Series if necessary
            if isinstance(trace_data, np.ndarray):
                trace_data = pd.Series(trace_data)
            if isinstance(timestamps, np.ndarray):
                timestamps = pd.Series(timestamps)
            
            # Apply detrend. For example: LOESS/LOWESS 
            trace_detrended = timeseries_detrend(trace_data, timestamps, method='lowess', window=300) 
            trace_detrended = pd.Series(trace_detrended)
            
            # Apply padding
            padded_detrended = timeseries_padding(trace_detrended, window_size)

            # Moving average in Pandas
            trace_rolling_avg = padded_detrended.rolling(window_size, center=True).mean()
            
            # Kernel smoothing with Gaussian window
            kernel_window = signal.windows.gaussian(window_size, std=std)
            # padded_trace_kernel = padded_detrended.rolling(window_size, center=True).mean()
            padded_trace_kernel = padded_detrended.rolling(window_size, win_type="gaussian", center=True).mean(std=std)
            
            # Trim the padded edges to match the original length
            trace_kernel = padded_trace_kernel[window_size // 2: -window_size // 2]
            timestamps_trimmed = timestamps[:len(trace_kernel)]  # Adjust timestamps length to match
            
            # Plotting
            fig, (ax1, ax2, ax3) = plt.subplots(1, 3, figsize=(20, 4), sharey=True)
            
            # Corrected signal
            ax1.set_title('Corrected Signal')
            ax1.plot(timestamps_trimmed, trace_detrended[:len(timestamps_trimmed)], label='Corrected Signal')
            ax1.set_xlabel('Timestamps')
            ax1.set_ylabel('Signal')
            ax1.legend()
            
            # Moving average
            ax2.set_title('Moving Average')
            ax2.plot(timestamps_trimmed, trace_rolling_avg[window_size // 2: -window_size // 2], label='Moving Average')
            ax2.set_xlabel('Timestamps')
            ax2.set_ylabel('Signal')
            ax2.legend()
            
            # Kernel smoothing
            ax3.set_title('Kernel Smoothing')
            ax3.plot(timestamps_trimmed, trace_kernel, label='Kernel Smoothing')
            
            # Plot the Gaussian window moving through the trace
            for start in range(window_size // 2, len(trace_detrended) - window_size // 2, window_size):
                end = start + window_size
                if end > len(trace_detrended):
                    end = len(trace_detrended)
                # Extract current window position
                current_kernel = kernel_window[:end-start]  # Adjust the kernel size to the window size
                kernel_time = timestamps_trimmed[start:end]  # Time stamps for the current window
                ax3.plot(kernel_time, current_kernel / current_kernel.max() * trace_kernel[start:end].max(), 
                         linestyle='--', color='black', alpha=0.5)
            
            ax3.set_xlabel('Timestamps')
            ax3.set_ylabel('Signal')
            ax3.legend()
            
            plt.tight_layout()
            
            # Save plots
            fig.savefig(f"{paths['plots']}/{filename}_smoothing_examples.tif", dpi=300)
            plt.show()